In [2]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from scipy.special import expit  


iris = load_iris()
X = iris.data
y = iris.target
class_labels = iris.target_names


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


X_train_b = np.c_[np.ones((X_train.shape[0], 1)), X_train]
X_test_b = np.c_[np.ones((X_test.shape[0], 1)), X_test]


def sigmoid(z):
    return expit(z)


def train_ovr_logistic_regression(X, y, lr=0.1, epochs=1000):
    n_samples, n_features = X.shape
    classes = np.unique(y)
    weights = np.zeros((len(classes), n_features))
    
    for i, c in enumerate(classes):
        y_binary = np.where(y == c, 1, 0)
        w = np.zeros(n_features)
        for _ in range(epochs):
            z = X.dot(w)
            y_pred = sigmoid(z)
            gradient = (1/n_samples) * X.T.dot(y_pred - y_binary)
            w -= lr * gradient
        weights[i] = w
    return weights


def predict_ovr(X, weights):
    probs = sigmoid(X.dot(weights.T))
    return np.argmax(probs, axis=1)


weights = train_ovr_logistic_regression(X_train_b, y_train, lr=0.1, epochs=2000)


y_pred = predict_ovr(X_test_b, weights)


acc = accuracy_score(y_test, y_pred)
print(f"Accuracy: {acc:.4f}\n")

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred), "\n")

print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=class_labels))


print("\nLearned weights (per class):")
for i, c in enumerate(class_labels):
    print(f"{c}: {weights[i]}")

Accuracy: 0.9000

Confusion Matrix:
[[10  0  0]
 [ 0  8  2]
 [ 0  1  9]] 

Classification Report:
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        10
  versicolor       0.89      0.80      0.84        10
   virginica       0.82      0.90      0.86        10

    accuracy                           0.90        30
   macro avg       0.90      0.90      0.90        30
weighted avg       0.90      0.90      0.90        30


Learned weights (per class):
setosa: [-2.48646266 -1.28559273  1.99283028 -2.42436318 -2.20906718]
versicolor: [-0.99943639  0.03685552 -1.37335258  1.39774629 -1.45878973]
virginica: [-4.20792339 -0.08738298 -0.419693    2.694882    4.24371129]
